In [1]:
import string

from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
def load_doc(filename):
    
    source = '/home/aman/Learnings/Datasets/nlp/'
    filename = source + filename
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


In [3]:
def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    # Remove punctuations
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [4]:
def save_doc(lines, filename):
    source = '/home/aman/Learnings/Datasets/nlp/'
    filename = source+filename
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [5]:
def preprocesstext(inFileName, outFileName):
        
    doc = load_doc(inFileName)
    print(doc[:200])
    
    tokens = clean_doc(doc)
    print(tokens[:200])
    print(f'Total tokens: {len(tokens)}')
    print(f'total unique tokens: {len(set(tokens))}')
    
    length = 50+1
    sequences = list()
    for i in range(length, len(tokens)):
        # select sequences of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
        
    print(sequences[:5])    
    print(f'Total Sequences: {len(sequences)}')
    
    save_doc(sequences, outFileName)

In [35]:
preprocesstext('TheRepublic.txt', 'TheRepublic_Sequences.txt')

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what
['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'c

In [6]:
doc = load_doc('TheRepublic_Sequences.txt')
lines = doc.split('\n')

In [7]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1

In [8]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes = vocab_size)
seq_length = X.shape[1]

In [9]:
print(seq_length)

50


In [62]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50 , input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            370500    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.fit(X, y, batch_size=128, epochs=300)
model.save('model.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Epoch 1/300
118633/118633 [==============================] - 97s 821us/step - loss: 2.2032 - acc: 0.4817
Epoch 2/300
118633/118633 [==============================] - 95s 804us/step - loss: 2.2378 - acc: 0.4766
Epoch 3/300
118633/118633 [==============================] - 104s 877us/step - loss: 2.2273 - acc: 0.4799
Epoch 4/300
118633/118633 [==============================] - 96s 806us/step - loss: 2.1862 - acc: 0.4879
Epoch 5/300
118633/118633 [==============================] - 146s 1ms/step - loss: 2.1515 - acc: 0.4927
Epoch 6/300
118633/118633 [==============================] - 123s 1ms/step - loss: 2.1338 - acc: 0.4957
Epoch 7/300
118633/118633 [==============================] - 148s 1ms/step - loss: 2.1207 - acc: 0.5003
Epoch 8/300
118633/118633 [==============================] - 145s 1ms/step - loss: 2.1113 - acc: 0.5001
Epoch 9/300
118633/118633 [==============================] - 145s 1ms/step - loss: 2.0981 - acc: 0.5022
Epoch 10/300
118633/118633 [==============================]

118633/118633 [==============================] - 137s 1ms/step - loss: 1.4164 - acc: 0.6446
Epoch 78/300
118633/118633 [==============================] - 105s 883us/step - loss: 1.6406 - acc: 0.5983
Epoch 79/300
118633/118633 [==============================] - 104s 874us/step - loss: 1.5372 - acc: 0.6250
Epoch 80/300
118633/118633 [==============================] - 103s 867us/step - loss: 1.4784 - acc: 0.6394
Epoch 81/300
118633/118633 [==============================] - 130s 1ms/step - loss: 1.4493 - acc: 0.6450
Epoch 82/300
118633/118633 [==============================] - 108s 914us/step - loss: 1.4748 - acc: 0.6389
Epoch 83/300
118633/118633 [==============================] - 110s 931us/step - loss: 1.4681 - acc: 0.6381
Epoch 84/300
118633/118633 [==============================] - 143s 1ms/step - loss: 1.4597 - acc: 0.6396
Epoch 85/300
118633/118633 [==============================] - 107s 902us/step - loss: 1.4472 - acc: 0.6443
Epoch 86/300
118633/118633 [============================

118633/118633 [==============================] - 109s 921us/step - loss: 1.4157 - acc: 0.6568
Epoch 154/300
118633/118633 [==============================] - 102s 858us/step - loss: 1.3733 - acc: 0.6604
Epoch 155/300
118633/118633 [==============================] - 104s 880us/step - loss: 1.2918 - acc: 0.6824
Epoch 156/300
118633/118633 [==============================] - 110s 925us/step - loss: 1.4235 - acc: 0.6611
Epoch 157/300
118633/118633 [==============================] - 111s 932us/step - loss: 1.5006 - acc: 0.6496
Epoch 158/300
118633/118633 [==============================] - 107s 900us/step - loss: 1.5210 - acc: 0.6476
Epoch 159/300
118633/118633 [==============================] - 107s 899us/step - loss: 2.0624 - acc: 0.5568
Epoch 160/300
118633/118633 [==============================] - 107s 901us/step - loss: 1.9931 - acc: 0.5406
Epoch 161/300
118633/118633 [==============================] - 107s 904us/step - loss: 2.0166 - acc: 0.5442
Epoch 162/300
118633/118633 [=============

KeyboardInterrupt: 

In [10]:
doc = load_doc('TheRepublic_Sequences.txt')
lines = doc.split('\n')
seq_length = len(lines[0].split())-1

In [11]:
model = load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [17]:
seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')

education furnish the best safeguard but they are welleducated already he replied i cannot be so confident my dear glaucon i said i am much more certain that they ought to be and that true education whatever that may be will have the greatest tendency to civilize and humanize them in



In [18]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    
    # generate a fixed number of words
    for _ in range(n_words):
        
        #encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        
        #truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        
        #predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
                
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [15]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

the metaphor and have to be called rarely things and the busybodies is defined down into the hands of the man and the neglect of the gods and the gods are bowed with fruit and the bed and the gods and not real however is nobody else fears and constituted
